<a href="https://colab.research.google.com/github/jhonrv1997/UPC/blob/main/PC1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica calificada 1

Profesor: [Carlos Adrián Alarcón](https://www.linkedin.com/in/carlos-adrian-alarcon-delgado/)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Parte teórica (7 ptos.)

Las preguntas tienen diferente nivel de dificultad. Respondan de manera precisa. Todos los dataset ya están cargados, así que ya no tienen que preocuparse por eso.


### Implementación de modelo de regresión logística haciendo uso de gradiente descendente

En el curso hemos conversado acerca del modelo de regresión logística y hemos explicado su funcionamiento en términos matemáticos. Ahora profundizaremos en ese aspecto. Para este ejercicio, intentaremos resolver el problema de clasificación mediante un modelo de regresión logística, estimando los parámetros a través la implementación del algoritmo de descenso de gradiente (este tema lo vamos a ver más adelante pero tómenlo como una buena introducción). Antes de iniciar, cabe recordar que partimos de que la variable de interés $Y$ se distribuye Bernoulli con parámetro p, donde p es la probabilidad de que $Y$ sea igual a 1, que en regresión logística se modela como:

$$ p = P(Y=1) = \frac{1}{1+exp\left(-(\beta_{0}+\beta_{1}X)\right)} = \frac{exp\left(\beta_{0} + \beta_{1}X\right)}{1+exp\left(\beta_{0} + \beta_{1}X\right)} $$

La estimación de los parámetros $\beta^{T}=\left[b, w_{1}, ...,w_{p}\right]$ que definen el hiperplano se hace maximizando el logaritmo de la función de máxima verosimilitud (maximum likelihood) de los datos:

\begin{equation} \arg \max_{\beta} \log \left(V \left(\beta \right) \right) = \arg \max_{\beta} \sum_{i=1}^{n} \log \left(p_{i}^{y_i}\left(1-p_{i}\right)^{1-y_i}\right) \end{equation}

En este caso, para simplificar la notación hacemos $p_i = \hat Y_i$. Dada esta función objetivo, nos hace falta definir ahora su gradiente, de modo que podamos proceder a implementar nuestro algortimo de descenso de gradiente para estimar los coeficientes. Aplicando propiedades de logaritmos:

$$ \sum_{i=1}^{n} \log\left(p_{i}^{y_i}\left(1-p_{i}\right)^{1-y_i}\right) = \sum_{i=1}^{n} \left(y_i\log\left(p_{i}\right) + \left(1-y_i\right)\log\left(1-p_{i}\right)\right) $$
$$ = \sum_{i=1}^{n} \left(y_i\log\left(p_{i}\right) + \log\left(1-p_{i}\right) - y_i\log\left(1-p_{i}\right)\right) = \sum_{i=1}^{n} \left(y_i\log\left(\frac{p_{i}}{1-p_{i}}\right) + \log\left(1-p_{i}\right)\right)$$

Es fácil demostrar que:

$$ \left(1-p\right) = \frac{1}{1+exp\left(\beta^{T}X\right)} $$

$$ \log\left(\frac{p}{1-p}\right) = \beta^{T}X $$

Por lo que, reemplazando, tenemos:

$$ \arg \max_{\beta} \sum_{i=1}^{n} \left(y_i\beta^{T}x_i  + \log\left(\frac{1}{1+exp\left(\beta^{T}x_i\right)}\right)\right) $$

Finalmente, nos queda derivar con respecto a cada parámetro, obteniendo:

$$ \frac {\partial \log \left(L \left(\beta \right) \right)}{\partial b} = \sum_{i=1}^{n} \left(y_i - p_{i}\right) $$

$$ \frac {\partial \log \left(L \left(\beta \right) \right)}{\partial w_{j}} = \sum_{i=1}^{n} \left((y_i - p_{i})x_{ij}\right) $$


En las siguientes funciones se implementa el modelo de regresión logística. No tienen que modificar nada, solamente revisar el resultado final, que es la estimación de los coeficientes.



In [2]:
# Definir función objetivo - logaritmo de la función de verosimilitud
def fobj(B, X, Y):
    suma = 0
    X=np.append(np.ones((len(X),1)), X, axis = 1) # Añadimos columna de 1s correspondiente al intercepto
    for i in range(len(X)):
        betaTx = np.matmul(np.transpose(B), X[i]) # Beta transpuesto por X
        suma = suma + Y[i]*betaTx + np.log(1/(1+np.exp(betaTx)))
    return suma

In [3]:
# Definir gradiente - vector con las derivadas parciales respecto a cada estimador
def gradiente(B, X, Y):
    gradiente = np.array([])
    X=np.append(np.ones((len(X),1)), X, axis = 1) # Añadimos columna de 1s correspondiente al intercepto
    for j in range(len(B)):
        suma = 0
        for i in range(len(X)):
            betaTx = np.matmul(np.transpose(B), X[i]) # Beta transpuesto por X
            pi = np.exp(betaTx)/(1+np.exp(betaTx))
            suma = suma + (Y[i]-pi)*X[i][j]
        gradiente=np.append(gradiente,suma) # Añadimos la derivada parcial con respecto a \beta_j
    return gradiente

In [4]:
# Función de descenso de gradiente
def regresion_logistica(fun_obj, gradiente, sol_inicial, step, tolerancia, X, Y):
    # Cargar solución inicial
    solucion = sol_inicial
    # Definir variable booleana para validar si se cumplió el criterio de parada
    stop = False
    # Definir contador de iteraciones
    num_iter = 1
    while stop==False:
        # Evaluar gradiente
        gradiente_eval = gradiente(solucion, X, Y)
        # Actualizar la solucion (se suma al tratarse de maximización)
        solucion = solucion + step * gradiente_eval
        # Evaluar la solucion
        solucion_eval = fun_obj(solucion, X, Y)
        # Actualizar contador de iteraciones
        num_iter += 1
        # Validar si se cumplió el criterio de parada, es decir, si la norma del gradiente evaluado en la solución actual es menor a la tolerancia dada
        if (np.linalg.norm(gradiente_eval) < tolerancia):
            stop = True
    print("Luego de "+str(num_iter)+" iteraciones, se encontró que los estimadores del modelo de regresión logística son "+str(solucion))
    return solucion

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/aladelca/machine_learning_model/refs/heads/main/archivos_trabajo/data%20(1).csv')
# Obtener Y
YTotal=np.array(df['Purchased'])

# Obtener X
XTotal=np.array(df[['EstimatedSalary']])
# Dividir muestra en train y test
XTrain, XTest, yTrain, yTest = train_test_split(XTotal, YTotal, test_size=0.33, random_state=0)
# Para facilitar convergencia, escalamos los datos
scaler = StandardScaler()
scaler.fit(XTrain)
XTrain=scaler.transform(XTrain)
XTest=scaler.transform(XTest)


# Definir solución inicial - estimadores en 0 - tenemos uno para cada variable independiente más el intercepto
sol_inicial = np.zeros(len(XTrain[1])+1)
# Definir tamaño de paso
step = 0.001
# Definir tolerancia
tolerancia = 0.001

# Llamar la función descenso de gradiente - tarda un poco
estimadores = regresion_logistica(fobj, gradiente, sol_inicial, step, tolerancia, XTrain, yTrain)
print(estimadores)

Luego de 236 iteraciones, se encontró que los estimadores del modelo de regresión logística son [-0.66575338  0.76767828]
[-0.66575338  0.76767828]


### Pregunta 1 (3 ptos.)

En función a los coeficiente estimados por el algoritmo de regresión lógistica estimado con el método de gradiente descendente, calcular las probabilidades correspondientes (no utilizar SKLearn). Asegurate de calcular las probabilidades de compra y no compra

In [ ]:
#### Escribe el código con la respuesta aquí ####




### Pregunta 2 (4 ptos.)

Ahora con los mismos datos, entrena el modelo usando `SKLearn` usando solamente la variable `EstimatedSalary` (deberías obtener parámetros bastante similares). Genera las predicciones usando el índice de Youden para hallar el punto de corte (por ende, las predicciones). Calcula el `accuracy`, `sensitivity` y `specificity`, además de mostrar la matriz de confusión. También calcula el AUC y muestra la curva ROC. Comenta los principales hallazgos del modelo.

In [ ]:
### Escribe el código con la respuesta aquí ###



Escribe tus hallazgos del modelo aquí

Otro método para encontrar el threshold indicado es utilizar la curva `precision recall`. La idea es encontrar el threshold que haga que las métricas `precision` y `recall` sean iguales (o lo más parecidas posible). La métrica `precision` la puedes calcular con la función `precision_score` de `SKLearn`. Identifica el mejor threshold en función a este método y compáralo con el que vimos en clase (y que has calculado en la primera parte de esta pregunta).

In [ ]:
### Escribe el código con la respuesta aquí ###


## Parte práctica (13 ptos.)

Muchas veces estamos en trabajos que no nos gustan y pensamos en renunciar, ¿alguna vez se han puesto a preguntar cómo identificar aquellos empleados que están a punto de renunciar `attrition`? Para eso tenemos el siguiente [dataset](https://www.kaggle.com/datasets/vjchoudhary7/hr-analytics-case-study/data) y lo que tenemos que predecir es justamente `attrition`.

### Pregunta 3 (2 ptos.)

Un paso importante de cualquier modelo de data science es el famoso `feature engineering` o la creación de variables. Justamente, por eso, a partir de los datasets indicados crea un dataset con al menos 15 variables. Estas pueden ser simplemente unir los datasets haciendo uso de `merge` o la creación de otras variables a partir de las existentes (campos calculados, etc.). Explica claramente porque la variable es relevante para cualquier modelo de machine learning tomando en cuenta el caso de uso (predicción de `attrition`):

In [ ]:
### Escribe el código aquí ###



Escribe tus conclusiones y respuesta aquí

### Pregunta 4 (3 ptos.)

Genera un modelo lineal de machine learning para predecir la variable `attrition`. Analiza los coeficientes e indica qué variable es la más importante. Si en la preparación de tus datos tienes datos nulos, puedos usar la imputación (`SimpleImputer`), solamente tendrías que tener en cuenta la separación `train.` y /`test` para evitar leakage (ya hemos hablado bastante en clase acerca de porque hacemos la separación). Además, escribe la fórmula lineal y evalúa el modelo escogiendo una métrica adecuada. Justifica la elección de métrica en función al caso de uso (respuestas genéricas sin conexión al caso de uso tendrán automáticamente 0)



In [ ]:
### Escribe el código aquí ###



Escribe tus conclusiones y respuesta aquí

### Pregunta 5 (3 ptos.)

Ahora entrena un modelo basado en `bagging`. Evalúalo con la métrica que escogiste en la pregunta anterior e indica qué modelo es mejor. Asimismo, explica qué es `bagging` y demuestra cómo es que está funcionando en tu modelo (mostrar unos ejemplos en código debería ser suficiente si es que sabes el concepto).

In [ ]:
### Escribe el código aquí ###



Escribe tus conclusiones y respuesta aquí

### Pregunta 6 (3 ptos.)

Ahora entrena un modelo de `boosting` y también evalúalo. Indica nuevamente qué modelo es mejor y analiza el último árbol (si sabes el concepto sabes al por qué hacemos esto), ¿qué conclusiones puedes hacer? (respuestas genéricas sin conexión al caso de uso tendrán automáticamente 0)

In [ ]:
### Escribe el código aquí ###

Escribe tus conclusiones y respuesta aquí

### Pregunta 7 (2 ptos.)

Ahora entrena un modelo `KNN` y calibra el hiperparámetro del número de vecinos para maximizar el rendimiento de la métrica que has escogido. Compáralo con el rendimiento de los otros modelos e indica cuál es mejor.

In [ ]:
### Escribe el código aquí ###

Escribe tus conclusiones y respuesta aquí